In [8]:
import os
import mlflow


In [13]:
# Set MLFLOW_HOME to the desired directory
os.environ['MLFLOW_HOME'] = "./.."

experiment_id = mlflow.create_experiment(
        name="testing_mlflow2",
        artifact_location="testing_mlflow2_artifacts",
        tags={"env": "dev", "version": "1.0.0"},
    )

print(experiment_id)


Exception: '/home/ubuntu/Bureau/OC/Projet5_oudelet_kevin/notebooks/mlruns' does not exist.

In [5]:
with mlflow.start_run(experiment_id=experiment_id):
    pass  # Do nothing in this run, just to create a run associated with the experiment

